In [6]:
from faker import Faker
import random
import time
import csv
import pandas as pd
import string 
import json 



In [7]:
faker = Faker()
# Load data from CSV
def load_csv(filename):
    return pd.read_csv(filename)
# Generate Unix timestamp
def generate_timestamp():
    return int(time.time())
# Write data to CSV
def write_to_csv(filename, fieldnames, data):
    with open(filename, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        if file.tell() == 0:
            writer.writeheader()
        writer.writerows(data)
# Function to generate nullable values
def nullable_value(generator, null_probability=0.2):
    if random.random() < null_probability:
        return None
    return generator()
# Unique value generator
def unique_value(existing_set, generator):
    value = generator()
    while value in existing_set:
        value = generator()
    existing_set.add(value)
    return value

In [ ]:
unique_location_id = set() 


unique_aircraft_id = set()
unique_registration = set()

unique_crew_member_id = set()   
unique_seat_id = set()
unique_flight_id = set()

In [ ]:
# Generate Airport data
#Bảng airport
current_airport_id = -1
def generate_airport_id():
    global current_airport_id
    current_airport_id += 1
    return current_airport_id
unique_iata = set()
unique_icao = set()
unique_airport_id = set()
location_data = load_csv('location.csv')
location_ids = list(location_data['LocationID'].unique())

# Hàm để lấy LocationID duy nhất
def get_unique_location_id():
    if not location_ids:
        raise ValueError("No more unique LocationIDs available.")
    return location_ids.pop(random.randrange(len(location_ids)))
def generate_airport(num_records):
    return [{
        'AirportID': uinque_value(unique_airport_id, generate_airport_id),
        'IATACode': unique_value(unique_iata, lambda: faker.bothify(text='???').upper()),
        'ICAOCode': nullable_value(lambda: unique_value(unique_icao, lambda: faker.bothify(text='????').upper())),
        'Name': faker.company() + ' Airport',
        'LocationID': get_unique_location_id(),
        'CreatedAt': generate_timestamp(),
        'UpdatedAt': generate_timestamp()
    } for _ in range(num_records)]
num_records = int(input('Enter the number of records to generate: '))
# Generate and write data
write_to_csv('airports.csv', generate_airport(1)[0].keys(), generate_airport(num_records))

In [ ]:
current_airlines_id = -1
def generate_airlines_id():
    global current_airlines_id
    current_airlines_id += 1
    return current_airlines_id
unique_icao_airline = set()
unique_iata = set()
def generate_airline(num_records):
    return [{
        'AirlineID': generate_airlines_id,
        'IATACode': unique_value(unique_iata, lambda: faker.bothify(text='??').upper()),
        'ICAOCode': nullable_value(lambda: unique_value(unique_icao_airline, lambda: faker.bothify(text='????').upper())),
        'Name': faker.company() + ' Airlines',
        'Country': faker.country(),
        'CreatedAt': generate_timestamp(),
        'UpdatedAt': generate_timestamp()
    } for _ in range(num_records)]
num_records = int(input('Enter the number of records to generate: '))
# Generate and write data
write_to_csv('airlines.csv', generate_airline(1)[0].keys(), generate_airline(num_records))

In [67]:
domains = ['gmail.com', 'yahoo.com', 'outlook.com', 'hotmail.com', 'icloud.com']
passenger_type = ['ADT', 'CHD', 'INF']
current_passenger_id = -1
def generate_passenger_id():
    global current_passenger_id
    current_passenger_id += 1
    return current_passenger_id
unique_email = set()
unique_passenger_id = set()
def generate_passenger(num_records):
    return [{
        'PassengerID' : unique_value(unique_passenger_id, generate_passenger_id),
        'FirstName': faker.first_name(),
        'LastName': faker.last_name(),
        'DateOfBirth': faker.date_of_birth().strftime('%Y-%m-%d'),
        'Gender': random.choice(['Female', 'Male']),
        'Email': unique_value(unique_email, lambda: faker.user_name() + '@' + random.choice(domains)),
        'PhoneNumber':  faker.phone_number(),
        'PassportNumber':faker.bothify(text='?########').upper(),
        'Nationality': faker.country(),
        'PassengerType': random.choice(passenger_type),
        'CreatedAt': generate_timestamp(),
        'UpdatedAt': generate_timestamp()
    } for _ in range(num_records)]
num_records = int(input('Enter the number of records to generate: '))
# Generate and write data
write_to_csv('passenger.csv', generate_passenger(1)[0].keys(), generate_passenger(num_records))


In [17]:
from datetime import datetime, timedelta
today = datetime.today().date()
reasons = [
    "Flight control systems malfunction",
    "Engine or sensors not working",
    "Navigation or communication equipment failure",
    "Corrosion on airframe or wings",
    "Worn-out tires or brakes",
    "Missing maintenance logs",
    "Expired airworthiness certificate",
    "Maintenance not done as scheduled",
    "Unauthorized structural modifications",
    "Installation of non-compliant equipment",
    "Improperly repaired after an incident",
    "Operation in restricted airspace",
    "Violation of weight or fuel regulations",
    "Aircraft owned by unauthorized entity",
    "Use for illegal activities (e.g. smuggling)"
]

status = ["Active"] + reasons
weights = [0.95] + [0.05 / len(reasons)] * len(reasons)

def generate_registration(num_records):
    unique_registrations = set()

    def generate_unique_registration():
        while True:
            reg_num = faker.bothify(text='??-########').upper()
            if reg_num not in unique_registrations:
                unique_registrations.add(reg_num)
                return reg_num

    records = []
    for _ in range(num_records):
        registered_date = faker.date_between(start_date='-30y', end_date='today')
        arc_expired_date = registered_date + timedelta(days=365 + random.randint(-60, 60))

        record = {
            'RegistrationNumber': generate_unique_registration(),
            'ReferenceCapacity': random.randint(100, 500),
            'MWOT': random.randint(100, 400),
            'RegisteredDate': registered_date.strftime('%Y-%m-%d'),
            'ARCExpiredDate': arc_expired_date.strftime('%Y-%m-%d'),
            'RegistrationStatus': random.choices(status, weights=weights, k=1)[0]
        }
        records.append(record)

    return records
num_records = int(input('Enter the number of records to generate: '))

data = generate_registration(num_records)

write_to_csv('Registration.csv', data[0].keys(), data)

print(f"{num_records} records have been written to Registration.csv")


5000 records have been written to Registration.csv


In [68]:
# Danh sách mã ICAO, tên model và nhà sản xuất được cung cấp
aircraft_data = """
A124, Antonov An-124 Ruslan
A140, Antonov An-140
A148, Antonov An-148
A158, Antonov An-158
A19N, Airbus A319neo
A20N, Airbus A320neo
A21N, Airbus A321neo
A225, Antonov An-225 Mriya
A306, Airbus A300-600
A30B, Airbus A300B2, A300B4 and A300C4
A310, Airbus A310-200
A318, Airbus A318
A319, Airbus A319
A320, Airbus A320
A321, Airbus A321
A332, Airbus A330-200
A333, Airbus A330-300
A337, Airbus A330-700 "BelugaXL"
A338, Airbus A330-800
A339, Airbus A330-900
A342, Airbus A340-200
A343, Airbus A340-300
A345, Airbus A340-500
A346, Airbus A340-600
A359, Airbus A350-900
A35K, Airbus A350-1000
A388, Airbus A380-800
A3ST, Airbus A300-600ST "Super Transporter" / "Beluga"
A400, Airbus A400M Atlas
A748, Hawker Siddeley HS 748
AC90, Gulfstream/Rockwell (Aero) Turbo Commander 690
AJ27, Comac ARJ21-700 / C909
AN12, Antonov An-12
AN24, Antonov An-24
AN26, Antonov An-26
AN28, Antonov An-28
AN30, Antonov An-30
AN32, Antonov An-32
AN72, Antonov An-72 / An-74
AT43, Aerospatiale/Alenia ATR 42-300 / 320
AT45, Aerospatiale/Alenia ATR 42-500
AT46, Aerospatiale/Alenia ATR 42-600
AT72, Aerospatiale/Alenia ATR 72-201/-202
AT73, Aerospatiale/Alenia ATR 72-211/-212
AT75, Aerospatiale/Alenia ATR 72-212A (500)
AT76, Aerospatiale/Alenia ATR 72-212A (600)
ATP, British Aerospace ATP
B190, Beechcraft 1900
B37M, Boeing 737 MAX 7
B38M, Boeing 737 MAX 8
B39M, Boeing 737 MAX 9
B3XM, Boeing 737 MAX 10
"""

import csv
import random

# Hàm sinh sức chứa dựa vào nhà sản xuất
def generate_capacity(manufacturer):
    if manufacturer in ['Airbus', 'Boeing']:
        return random.randint(150, 400)
    elif manufacturer in ['Antonov', 'Lockheed']:
        return random.randint(50, 250)
    elif manufacturer in ['Comac', 'ATR', 'Beechcraft', 'Hawker']:
        return random.randint(30, 100)
    elif manufacturer in ['Gulfstream', 'Dassault', 'Cessna']:
        return random.randint(10, 30)
    else:
        return random.randint(20, 200)

# Dictionary để lưu kết quả
aircraft_lookup = {}

# Xử lý dữ liệu và đưa vào dictionary
for line in aircraft_data.strip().split("\n"):
    parts = line.split(", ")
    if len(parts) == 2:
        icao = parts[0].strip()
        model = parts[1].strip()
        manufacturer = model.split()[0]  # Lấy từ đầu tiên làm tên nhà sản xuất
        capacity = generate_capacity(manufacturer)  # Sinh sức chứa
        aircraft_lookup[icao] = (model, manufacturer, capacity)

# Ghi kết quả ra file CSV
csv_filename = "aircraft_lookup.csv"
with open(csv_filename, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["ICAO", "Model", "Manufacturer", "Capacity"])
    for icao, (model, manufacturer, capacity) in aircraft_lookup.items():
        writer.writerow([icao, model, manufacturer, capacity])

csv_filename



'aircraft_lookup.csv'

In [32]:
# Load aircraft data from the uploaded CSV file
aircraft_data = load_csv('aircraft_lookup.csv')

# Convert the data into a list of tuples (ICAO, Model, Manufacturer)
aircraft_lookup = list(aircraft_data[['ICAO', 'Model', 'Manufacturer', 'Capacity']].itertuples(index=False, name=None))

In [33]:
# Load aircraft data from the uploaded CSV file
airlinesID_data = load_csv('airlines.csv')

# Convert the data into a list of tuples (AirlinesID)
airlines_lookup = list(airlinesID_data[['AirlineID']].itertuples(index=False, name=None))

In [ ]:
# Aircraft Table
def generate_aircraft(num_records):
    return [{
        'AircraftID': unique_value(unique_aircraft_id, g),
        'RegistrationNumber': unique_value(unique_registration, lambda: faker.bothify(text='??-####').upper()),
        'ICAOTypeCode': (icao := random.choice(aircraft_lookup))[0],
        'Model': icao[1],
        'Manufacturer': icao[2],
        'Capacity':icao[3],
        'Status': random.choice(['Active', 'Maintenance']),
        'OwnerAirlineID': (airid := random.choice(airlines_lookup))[0], # Placeholder for FK to Airline
        'DefaultConfigurationCode': faker.bothify(text='CFG####').upper(),
        'CreatedAt': nullable_value(lambda: generate_timestamp()),
        'UpdatedAt': nullable_value(lambda: generate_timestamp())
    } for _ in range(num_records)]
num_records = int(input('Enter the number of records to generate: '))
# Generate and write data
write_to_csv('aircraft.csv', generate_aircraft(1)[0].keys(), generate_aircraft(num_records))
# Passenger Table

In [43]:
def generate_crewMember(num_records):
    return [{
        'CrewMemberID': unique_value(unique_crew_member_id, lambda: faker.bothify(text='CM########')),
        'FirstName': faker.first_name(),
        'LastName': faker.last_name(),
        'Role': random.choices(
            ['Captain', 'First Officer', 'Purser', 'Flight Attendant'],
            weights=[1, 2, 3, 4],  # Tỷ lệ tương ứng
            k=1  # Chọn 1 giá trị
            )[0],
        'AirlineID': (airid := random.choice(airlines_lookup))[0], 
        'LicenseNumber': faker.bothify(text='LIC#########'),
        'IsActive': random.choice([True, False]),
        'CreatedAt': nullable_value(lambda: generate_timestamp()),
        'UpdatedAt': nullable_value(lambda: generate_timestamp())
    } for _ in range(num_records)]
num_records = int(input('Enter the number of records to generate: '))
# Generate and write data
write_to_csv('crewMem.csv', generate_crewMember(1)[0].keys(), generate_crewMember(num_records))

In [47]:
# Load aircraft data from the uploaded CSV file
aircraftID_data = load_csv('aircraft.csv')

# Convert the data into a list of tuples (AircraftID)
aircraftID_lookup = list(aircraftID_data[['AircraftID']].itertuples(index=False, name=None))

In [62]:
# Hàm tạo dữ liệu AircraftSeat
seat_types = ['Window', 'Aisle', 'Middle', 'Exit Row', 'Bulkhead']
cabin_classes = ['Economy','Premium economy','Business class', 'First class']
characteristics = [
'Extra Legroom', 'Recline', 'Adjustable Headrest', 'Lumbar Support', 'Massage Function', 'Footrest',
'Power Outlet', 'USB Port', 'Personal TV', 'Wi-Fi Access', 'Audio Jack', 'Bluetooth Connectivity',
'In-seat Dining Table', 'Personal Reading Light', 'Coat Hook', 'Storage Pocket', 'Call Button', 
'Cup Holder', 'Adjustable Tray Table', 'Seatbelt Extender', 'Baby Bassinet Option', 
'Accessibility Assistance', 'Oxygen Mask', 'Privacy Divider', 'Noise-Cancelling Headphones', 
'Window Shade', 'Mood Lighting']
def generate_aircraft_seat(num_records):
    seats = []
    for _ in range(num_records):
        # Chọn các đặc điểm ngẫu nhiên
        seat_characteristics = random.sample(characteristics, random.randint(1, 5))
        
        seat = {
            'SeatID': unique_value(unique_seat_id, lambda: faker.bothify(text='ST########')),
            'AircraftID': random.choice(aircraftID_lookup)[0],
            'ConfigurationCode': faker.bothify(text='Y########'),
            'SeatNumber': f"{random.randint(1, 100)}{random.choice(['A', 'B', 'C', 'D', 'E', 'F'])}",
            'SeatType': random.choice(seat_types),
            'CabinClass': random.choice(cabin_classes),
            'Characteristics': json.dumps(seat_characteristics),  # Định dạng JSON đúng
            'IsBlocked': random.choices([True, False], weights=[10, 90], k=1)[0],
            'IsExitRow': random.choices([True, False], weights=[15, 85], k=1)[0],
            'CreatedAt': generate_timestamp(),
            'UpdatedAt': generate_timestamp()
        }
        seats.append(seat)
    return seats
    
# Số lượng bản ghi cần tạo
num_records = int(input('Enter the number of AircraftSeat records to generate: '))

# Ghi vào CSV
write_to_csv('aircraft_seat.csv', generate_aircraft_seat(1)[0].keys(), generate_aircraft_seat(num_records))
